In [1]:
import numpy as np
from scipy.io import arff
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn import metrics
import scipy as sp
import logging
from sklearn.neighbors import NearestNeighbors
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
    
def run_lof(X, y, k=60):
    clf = LocalOutlierFactor(n_neighbors=k)
    clf.fit(X)
    lof_scores = -clf.negative_outlier_factor_
    return lof_scores

def get_predictions(scores, num_outliers = 400, method_name = 'LOF'):
    threshold = np.sort(scores)[::-1][num_outliers]
    # threshold, max_f1 = get_best_f1_score(y, lof_scores)
    predictions = np.array(scores > threshold)
    predictions = np.array([int(i) for i in predictions])
#     print('F1 for {} : {}'.format(method_name, metrics.f1_score(y, predictions)))
    return predictions, scores, metrics.f1_score(y, predictions)

def get_best_F1(scores):
    best_f1 = 0
    for i in range(np.shape(scores)[0]):
        threshold = np.sort(scores)[::-1][i]
        predictions = np.array(scores > threshold)
        predictions = np.array([int(i) for i in predictions])
        cur_f1 = metrics.f1_score(y, predictions)
        best_f1 = max(cur_f1, best_f1)
    return best_f1

def run_knn(X, y, k=60):
    neigh = NearestNeighbors(n_neighbors=k)
    neigh.fit(X)
    knn_dists = neigh.kneighbors(X)[0][:,-1]
    return knn_dists

def run_isolation_forest(X, y, max_features = 1.0):
    # training the model
    clf = IsolationForest(random_state=42,max_features=max_features)
    clf.fit(X)
    # predictions
    sklearn_score_anomalies = clf.decision_function(X)
    if_scores = [-1*s + 0.5 for s in sklearn_score_anomalies]
    return if_scores

def mahalanobis(x):
    """Compute the Mahalanobis Distance between each row of x and the data
    """
    x_minus_mu = x - np.mean(x)
    cov = np.cov(x.T)
    inv_covmat = sp.linalg.inv(cov)
    results = []
    x_minus_mu = np.array(x_minus_mu)
    for i in range(np.shape(x)[0]):
        cur_data = x_minus_mu[i,:]
        results.append(np.dot(np.dot(x_minus_mu[i,:], inv_covmat), x_minus_mu[i,:].T))
    return np.array(results)
#     left_term = np.dot(x_minus_mu, inv_covmat)
#     mahal = np.dot(left_term, x_minus_mu.T)
#     print(mahal.diagonal())
#     return mahal.diagonal()

def run_mahalanobis(X, y):
    # training the model
    dist = mahalanobis(x=X)
    return dist

def load_dataset(filename):
    with open(filename, 'r') as f:
        data, meta = arff.loadarff(f)
    data = pd.DataFrame(data)
    X = data.drop(columns=['id', 'outlier'])
    # Map dataframe to encode values and put values into a numpy array
    y = data["outlier"].map(lambda x: 1 if x == b'yes' else 0).values
    return X, y

### Load SpamBase dataset

In [ ]:
filename = './SpamBase_withoutdupl_norm_40.arff'
X, y = load_dataset(filename=filename)
print(np.shape(X), np.shape(y))
K = 9
N = 1679
class_balance = [1- N/4207.0, N/4207.0]
lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
# mahalanobis_N_range=[N]
mahalanobis_N_range = [1400, 1500, 1600, 1700, 1800, 1900]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range * 10) 
print(N/len(y))

### Load pageblock dataset

In [2]:
filename = './PageBlocks/PageBlocks_norm_10.arff'
X, y = load_dataset(filename=filename)
print(np.shape(X), np.shape(y))
K = 80
N = 560
# num_outliers = [N, N, N, N]
class_balance = [0.9, 0.1]
# lof_krange = [55, 60, 65, 70, 75] 
# knn_krange = [55, 60, 65, 70, 75] 
# if_range = [0.5, 0.6,0.7, 0.8,0.9]
lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
# lof_krange = range(70,90,4) 
# knn_krange = [60, 70, 80, 90, 100] 
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
# mahalanobis_N_range=[560]
mahalanobis_N_range = [300, 400, 500, 600, 700, 800]
# mahalanobis_N_range = [550, 560, 570, 580, 590, 600]
N_size = 6
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range * 10)

(5473, 10) (5473,)


### Load Hepatitis dataset

In [ ]:
filename = './Hepatitis_withoutdupl_norm_16.arff'
X, y = load_dataset(filename=filename)
print(np.shape(X), np.shape(y))
N = 19
print(sum(y))
class_balance = [1- N/80.0, N/80.0]
lof_krange = list(range(2,22,2)) * 6
knn_krange = list(range(2,22,2)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6 
mahalanobis_N_range=[N]
mahalanobis_N_range = [10, 12, 15, 18, 20, 25]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range * 10)

### Load Pima dataset

In [ ]:
filename = './Pima_withoutdupl_norm_35.arff'
X, y = load_dataset(filename=filename)
print(np.shape(X), np.shape(y))
K = 100
N = 268
print(N/len(y))
num_outliers = [N, N, N, N]
class_balance = [1- N/768.0, N/768.0]
lof_krange = list(range(10,210,10)) * 6
knn_krange = list(range(10,210,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
mahalanobis_N_range=[N]
mahalanobis_N_range = [220,230,240,250,260,270]

if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range * 20)

### ALOI dataset

In [ ]:
filename = './ALOI_withoutdupl_norm.arff'
X, y = load_dataset(filename=filename)
print(np.shape(X), np.shape(y))
print(sum(y))
N = 1508
num_outliers = [N, N, N, N]
class_balance = [1- N/49534.0, N/49534.0]
lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
# mahalanobis_N_range=[N]
mahalanobis_N_range=[1500, 2000, 2500, 3000, 3500, 4000]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)

### Load InternetAds

In [ ]:
filename = './InternetAds_withoutdupl_norm_19.arff'
X, y = load_dataset(filename=filename)
print(np.shape(X), np.shape(y))
print(sum(y))
N = 368
num_outliers = [N, N, N, N]
class_balance = [1- N/1966.0, N/1966.0]
lof_krange = list(range(5,55,5)) * 6
knn_krange = list(range(5,55,5)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
# mahalanobis_N_range=[N]
mahalanobis_N_range = [300, 350, 400, 450, 500, 550]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)

### Load KDDCup 99 dataset

In [ ]:
filename = './KDDCup99_withoutdupl_norm_catremoved.arff'
X, y = load_dataset(filename=filename)
print(np.shape(X), np.shape(y))
print(sum(y))
N = 200
num_outliers = [N, N, N, N]
class_balance = [1- N/48113.0, N/48113.0]
lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
mahalanobis_N_range=[500,1000,1500,2000,2500,3000]
# mahalanobis_N_range = [20, 40, 60,80, 100,120]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)

In [ ]:
filename='kdd99-unsupervised-ad.csv'
import pandas as pd
data = pd.read_csv(filename, header=None)
X = data.drop(columns=[29])
print(np.shape(np.array(X)))
# Map dataframe to encode values and put values into a numpy array
y = data[29].map(lambda x: 1 if x == 'o' else 0).values
print(sum(y))

lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
mahalanobis_N_range=[1000,1500,2000,2500,3000,3500]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)

### Load shuttle dataset

In [ ]:
import hdf5storage
mat = hdf5storage.loadmat('shuttle.mat')
X = mat['X']
y = mat['y']
print(len(y))
print(np.sum(y)/len(y))
lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
mahalanobis_N_range=[1000,1500,2000, 2500,3000, 3500]
# mahalanobis_N_range = [20, 40, 60,80, 100,120]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)
print(np.shape(X))

# normalize
# from sklearn.preprocessing import Normalizer
# transformer = Normalizer().fit(X) 
# X = transformer.transform(X)

### Load mulcross dataset

In [ ]:
filename = './mulcross.arff'

with open(filename, 'r') as f:
    data, meta = arff.loadarff(f)
data = pd.DataFrame(data)
X = data.drop(columns=['Target'])
y = data["Target"].map(lambda x: 1 if x == b'Anomaly' else 0).values
# X, y = load_dataset(filename=filename)
print(np.shape(X), np.shape(y))
print(sum(y))
print(sum(y)/len(y))
lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
mahalanobis_N_range=[20000, 22000, 24000, 26000, 28000, 30000]
# mahalanobis_N_range = [20, 40, 60,80, 100,120]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)

### Load HTTP dataset

In [ ]:
import hdf5storage
mat = hdf5storage.loadmat('http.mat')
X = mat['X']
y = mat['y']
print(len(y))
print(np.sum(y))
print(np.sum(y)/len(y))
lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
# mahalanobis_N_range=[1500, 2000, 2500, 3000, 3500, 4000]
mahalanobis_N_range=[5000, 10000, 15000,20000, 25000, 30000]
# mahalanobis_N_range=[10000, 15000, 20000, 25000, 30000, 35000]
# mahalanobis_N_range = [20, 40, 60,80, 100,120]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)

# # remove duplicates
# newdata = pd.DataFrame(np.concatenate((X,y), axis = 1)).drop_duplicates()
# X = newdata[[0,1,2]].values
# y = np.array([1 if i==1.0 else 0 for i in newdata[[3]].values])
# print('Remove duplicates: ', len(y))

# # normalize
# from sklearn.preprocessing import Normalizer
# transformer = Normalizer().fit(X) 
# X = transformer.transform(X)

### Load ForestCover Dataset

In [ ]:
import hdf5storage
import pickle
dataset = pickle.load(open("cover_dataset.pickle", "rb"))
X = dataset['X']
y = dataset['y']
print(np.sum(y))
print(len(y))
lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
mahalanobis_N_range=[5000, 8000, 10000, 12000, 15000, 18000]
# mahalanobis_N_range = [20, 40, 60,80, 100,120]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)

from sklearn.preprocessing import RobustScaler
transformer = RobustScaler().fit(X)
X_transformed = transformer.transform(X)

### Load Annthyroid dataset

In [ ]:
filename = './Annthyroid_withoutdupl_norm_07.arff'
X, y = load_dataset(filename=filename)
print(np.shape(X), np.shape(y))
print(sum(y)/len(y))
N = 534
num_outliers = [N, N, N, N]
class_balance = [1- N/7129.0, N/7129.0]
lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
# mahalanobis_N_range=[N]
mahalanobis_N_range=[300, 400,500,600,700,800]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)

In [ ]:
import hdf5storage
mat = hdf5storage.loadmat('smtp.mat')
X = mat['X']
y = mat['y']
print(len(y))
print(np.sum(y))

lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
# mahalanobis_N_range=[20, 40, 60, 80, 100, 120]
mahalanobis_N_range = [50, 100, 150, 200, 250, 300]
# mahalanobis_N_range = [200, 400, 600, 800, 1000, 1200]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)

In [ ]:
import hdf5storage
mat = hdf5storage.loadmat('cardio.mat')
X = mat['X']
y = mat['y']
print(len(y))
print(np.sum(y)/len(y))
lof_krange = list(range(10,110,10)) * 6
knn_krange = list(range(10,110,10)) * 6
if_range = [0.5, 0.6, 0.7, 0.8, 0.9] * 6
mahalanobis_N_range=[150,200,250,300,350,400]
# mahalanobis_N_range = [20, 40, 60,80, 100,120]
if_N_range = np.sort(mahalanobis_N_range * 5)
N_range = np.sort(mahalanobis_N_range *10)
print(np.shape(X))

In [3]:
all_results = []
all_scores = []
f1s = []

temp_lof_results = dict()
unique_lof_ks = list(set(lof_krange)) 
for k in unique_lof_ks:
#     print(k)
    lof_scores = run_lof(X, y, k=k)
    temp_lof_results[k] = lof_scores
for i in range(len(lof_krange)):
    lof_predictions, lof_scores, f1 = get_predictions(temp_lof_results[lof_krange[i]], num_outliers=N_range[i], method_name='LOF')
    all_results.append(lof_predictions)
    all_scores.append(lof_scores)
    f1s.append(f1)
best_lof_f1 = 0
for i in np.sort(unique_lof_ks):
    temp_f1 = max(np.array(f1s[0:60])[np.where(np.array(lof_krange) == i)[0]])
    print('LOF k = {}, best F-1 = {}'.format(i, temp_f1))
    best_lof_f1 = max(best_lof_f1, temp_f1)
print('Best LOF F-1 = {}'.format(best_lof_f1))

temp_knn_results = dict()
unique_knn_ks = list(set(knn_krange)) 
for k in unique_knn_ks:
    print(k)
    knn_scores = run_knn(X, y, k=k)
    temp_knn_results[k] = knn_scores
for i in range(len(knn_krange)):
    knn_predictions, knn_scores,f1 = get_predictions(temp_knn_results[knn_krange[i]], num_outliers=N_range[i], method_name='KNN')
    all_results.append(knn_predictions)
    all_scores.append(knn_scores)
    f1s.append(f1)
best_knn_f1 = 0
for i in np.sort(unique_knn_ks):
    temp_f1 = max(np.array(f1s[60:120])[np.where(np.array(knn_krange) == i)[0]])
    print('KNN k = {}, best F-1 = {}'.format(i, temp_f1))
    best_knn_f1 = max(best_knn_f1, temp_f1)
print('Best KNN F-1 = {}'.format(best_knn_f1))
    
temp_if_results = dict()
unique_if_features = list(set(if_range)) 
for k in unique_if_features:
    print(k)
    if_scores = run_isolation_forest(X, y, max_features=k)
    temp_if_results[k] = if_scores
for i in range(len(if_range)):
    if_predictions, if_scores,f1 = get_predictions(temp_if_results[if_range[i]], num_outliers=N_range[i], method_name='IF')
    all_results.append(if_predictions)
    all_scores.append(if_scores)
    f1s.append(f1)
best_if_f1 = 0
for i in np.sort(unique_if_features):
    temp_f1 = max(np.array(f1s[120:150])[np.where(np.array(if_range) == i)[0]])
    print('IF = {}, best F-1 = {}'.format(i, temp_f1))
    best_if_f1 = max(best_if_f1, temp_f1)
print('Best IF F-1 = {}'.format(best_if_f1))
    
mahalanobis_scores = run_mahalanobis(X, y)
best_mahala_f1 = 0
for i in range(len(mahalanobis_N_range)):
    mahalanobis_predictions,mahalanobis_scores,f1 = get_predictions(mahalanobis_scores, num_outliers=mahalanobis_N_range[i], method_name='mahala')
    all_results.append(mahalanobis_predictions)
    all_scores.append(mahalanobis_scores)
    best_mahala_f1 = max(best_mahala_f1, f1)
    f1s.append(f1)
print('mahalanobis = {}'.format(max(np.array(f1s[150:]))))
print('Best Mahala F-1 = {}'.format(best_mahala_f1))
L = np.stack(all_results).T
scores = np.stack(all_scores).T

LOF k = 10, best F-1 = 0.39999999999999997
LOF k = 20, best F-1 = 0.4520833333333334
LOF k = 30, best F-1 = 0.42499999999999993
LOF k = 40, best F-1 = 0.4547169811320755
LOF k = 50, best F-1 = 0.49056603773584906
LOF k = 60, best F-1 = 0.5020833333333333
LOF k = 70, best F-1 = 0.5018867924528301
LOF k = 80, best F-1 = 0.5120689655172415
LOF k = 90, best F-1 = 0.4862068965517241
LOF k = 100, best F-1 = 0.4773584905660378
Best LOF F-1 = 0.5120689655172415
100
70
40
10
80
50
20
90
60
30
KNN k = 10, best F-1 = 0.39841269841269844
KNN k = 20, best F-1 = 0.42698412698412697
KNN k = 30, best F-1 = 0.42698412698412697
KNN k = 40, best F-1 = 0.43333333333333335
KNN k = 50, best F-1 = 0.4317460317460317
KNN k = 60, best F-1 = 0.42758620689655175
KNN k = 70, best F-1 = 0.4241379310344828
KNN k = 80, best F-1 = 0.4190476190476191
KNN k = 90, best F-1 = 0.4174603174603174
KNN k = 100, best F-1 = 0.41764705882352937
Best KNN F-1 = 0.43333333333333335
0.5
0.6
0.8
0.9
0.7
IF = 0.5, best F-1 = 0.409433

In [4]:
print(np.shape(L))
print(np.shape(scores))

(5473, 156)
(5473, 156)


### Majority Vote

In [5]:
mid = np.shape(L)[1]/2
predictions = np.full((len(y)), 0)
predictions[np.sum(L, axis = 1) > mid] = 1
print('F1 for MV:', metrics.f1_score(y, predictions))

F1 for MV: 0.43640124095139604


### Save and load pickles

In [6]:
L_prev = L
scores_prev = scores

In [13]:
L = L_prev
scores = scores_prev

In [14]:
# print(max(f1s)) 
print(np.shape(L))

(5473, 156)


In [15]:
prediction_result_list = []
classifier_result_list = []
prediction_list = []
cur_f1_scores = []
prediction_high_conf_outliers = np.array([])
prediction_high_conf_inliers = np.array([])
prediction_classifier_disagree = np.array([])

In [16]:
index_range = np.array([[0, 60], [60, 120], [120, 150], [150, 156]])
coef_index_range = np.array([[0, 10], [10, 20], [20, 25], [25, 26]])
coef_remain_index = range(156)

In [17]:
# index_range = np.array([[0, 120], [120, 240], [240, 270], [270, 276]])
# coef_index_range = np.array([[0, 20], [20, 40], [40, 45], [45, 46]])
# coef_remain_index = range(276)

In [18]:
scores_for_training_indexes = []
for i in range(len(index_range)):
    start=index_range[i][0]
    temp_range = coef_index_range[i][1]-coef_index_range[i][0]
    scores_for_training_indexes  = scores_for_training_indexes + list(range(start, start+temp_range))
print(scores_for_training_indexes) 
scores_for_training = scores[:, np.array(scores_for_training_indexes)]
print(np.shape(scores_for_training))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 120, 121, 122, 123, 124, 150]
(5473, 26)


### Iterative train LR and classifier(SVM)

In [19]:
# stable version
high_confidence_threshold = 0.99
low_confidence_threshold = 0.01
LR_threshold = 0.5
max_iter = 200
remain_params_tracking = np.array(range(0,np.max(coef_index_range)))
training_data_F1 = []
two_prediction_corr = []

min_max_diff = []
N_size = 6

last_training_data_indexes = []
counter = 0

for i_range in range(0, 50):
    print("##################################################################")
    print('Iteration = {}, L shape = {}'.format(i_range, np.shape(L)))
    num_methods = np.shape(L)[1]
    
    agree_outlier_indexes = np.sum(L,axis=1)==np.shape(L)[1]
#     print('All agree, Number of outliers = {}'.format(sum(agree_outlier_indexes)))
    agree_inlier_indexes = np.sum(L,axis=1)==0
#     print('All agree, Number of inliers = {}'.format(sum(agree_inlier_indexes)))

    disagree_indexes = np.where(np.logical_or(np.sum(L,axis = 1)==0, np.sum(L,axis = 1)==num_methods)==0)[0]
    
#     all_inlier_indexes = np.where(agree_inlier_indexes)[0]
    all_inlier_indexes = np.setdiff1d(np.where(agree_inlier_indexes)[0], prediction_high_conf_outliers)
    if len(prediction_high_conf_inliers) >0:
        all_inlier_indexes = np.intersect1d(np.setdiff1d(np.where(agree_inlier_indexes)[0], prediction_high_conf_outliers), prediction_high_conf_inliers)
#     print('num of inliers = {}'.format(np.shape(all_inlier_indexes)[0]))
#     all_outlier_indexes = np.union1d(np.where(agree_outlier_indexes)[0], self_agree_index_list)

#     if(len(np.intersect1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)) > 0 and
#       (len(np.union1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)) > 2000)):
#         all_outlier_indexes = np.intersect1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)
#     else:
    all_outlier_indexes = np.union1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)
#     if(len(all_outlier_indexes) > 1000):
#         all_outlier_indexes = np.random.RandomState(1).permutation(all_outlier_indexes)[:1000]
        
#     all_outlier_indexes = np.union1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)
#     print('num of outliers = {}'.format(np.shape(all_outlier_indexes)[0]))
    all_inlier_indexes = np.setdiff1d(all_inlier_indexes, prediction_classifier_disagree)
    
    self_agree_index_list = []
    if((len(all_outlier_indexes) == 0) or (len(all_inlier_indexes)/ len(all_outlier_indexes) > 1000)):
        for i in range(0, len(index_range)):
            if(index_range[i,1]-index_range[i,0] <= 6):
                continue
            temp_index = disagree_indexes[np.where(np.sum(L[disagree_indexes][:,index_range[i,0]: index_range[i,1]], axis = 1)==(index_range[i,1]-index_range[i,0]))[0]]
            self_agree_index_list = np.union1d(self_agree_index_list, temp_index)
        self_agree_index_list = [int(i) for i in self_agree_index_list]
#     self_agree_index_list = np.random.RandomState(1).permutation(self_agree_index_list)[:500]
    all_outlier_indexes = np.union1d(all_outlier_indexes, self_agree_index_list)
    all_outlier_indexes = np.setdiff1d(all_outlier_indexes, prediction_classifier_disagree)
    print('num of outliers = {}'.format(np.shape(all_outlier_indexes)[0]))
    
    
    from sklearn.preprocessing import RobustScaler
    from sklearn.preprocessing import StandardScaler
    data_indexes = np.concatenate((all_inlier_indexes, all_outlier_indexes), axis = 0)
    data_indexes = np.array([int(i) for i in data_indexes])
    labels = np.concatenate((np.zeros(len(all_inlier_indexes)), np.ones(len(all_outlier_indexes))), axis = 0)
    transformer = RobustScaler().fit(scores_for_training)
    scores_transformed = transformer.transform(scores_for_training)
    training_data = scores_transformed[data_indexes]
    print('Training data shape: ', np.shape(training_data))
    training_data_F1.append(metrics.f1_score(y[data_indexes], labels))
    print('Training data F-1', metrics.f1_score(y[data_indexes], labels))
    
    transformer = RobustScaler().fit(X)
    X_transformed = transformer.transform(X)
    X_training_data = X_transformed[data_indexes]
    print(np.shape(X_training_data))
    print(np.shape(labels))
    
    from sklearn.svm import SVC
    clf_X = SVC(gamma='auto', probability=True, random_state=0)
    clf_X.fit(X_training_data, labels)
    clf_predictions_X = clf_X.predict(X_transformed)
    clf_predict_proba_X = clf_X.predict_proba(X_transformed)[:,1]
    print("F-1 score from SVM:",metrics.f1_score(y, np.array([int(i) for i in clf_predict_proba_X > 0.5])))
    cur_f1_scores.append(metrics.f1_score(y, np.array([int(i) for i in clf_predict_proba_X > 0.5])))
    print('Number of outliers by SVM:', sum(clf_predictions_X))
    
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression(random_state=0, penalty='l2', max_iter=max_iter).fit(training_data, labels) 
    clf_predictions = clf.predict(scores_transformed)
    clf_predict_proba = clf.predict_proba(scores_transformed)[:,1]
#     LR_threshold = np.array(np.sort(clf_predict_proba)[::-1])[int(sum(clf_predictions_X))]
    print("F-1 score from LR:",metrics.f1_score(y, np.array([int(i) for i in clf_predict_proba > LR_threshold])))
    print('Number of outliers by LR: ', sum(np.array([int(i) for i in clf_predict_proba > LR_threshold])))
    
    agreed_outlier_indexes = np.where(np.sum(L,axis=1)==np.shape(L)[1])[0]
    agreed_inlier_indexes = np.where(np.sum(L,axis=1)==0)[0]
        
    prediction_result_list.append(clf_predict_proba)
    classifier_result_list.append(clf_predict_proba_X)
    
    prediction_list.append(np.array([int(i) for i in clf_predictions]))
    
    prediction_high_conf_outliers = np.intersect1d(np.where(prediction_result_list[-1] > high_confidence_threshold)[0],
                                                   np.where(classifier_result_list[-1] > high_confidence_threshold)[0])
#     print('length of prediction_high_conf_outliers:' , len(prediction_high_conf_outliers))
    prediction_high_conf_inliers = np.intersect1d(np.where(prediction_result_list[-1] < low_confidence_threshold)[0],
                                                   np.where(classifier_result_list[-1] < low_confidence_threshold)[0])
#     print('length of prediction high conf inliers: ', len(prediction_high_conf_inliers))
    
    temp_prediction = np.array([int(i) for i in prediction_result_list[-1] > LR_threshold])
    temp_classifier = np.array([int(i) for i in classifier_result_list[-1] > 0.5])
    prediction_classifier_disagree = np.where(temp_prediction != temp_classifier)[0]
#     print('length of prediction-classifier disagree: {}'.format(len(prediction_classifier_disagree)))
#     print('length of prediction-classifier disagree in training: {}'.format(len(np.where(temp_prediction[data_indexes] != temp_classifier[data_indexes])[0])))
    print(np.corrcoef(clf_predict_proba,clf_predict_proba_X))
    two_prediction_corr.append(np.corrcoef(clf_predict_proba,clf_predict_proba_X)[0,1])

    if np.max(coef_index_range) >= 2:
#         if(len(prediction_high_conf_outliers) > 0 and len(prediction_high_conf_inliers) > 0):
#             new_data_indexes = np.concatenate((prediction_high_conf_outliers, prediction_high_conf_inliers), axis = 0)
#             new_data_indexes = np.array([int(i) for i in new_data_indexes])
#             new_labels = np.concatenate((np.ones(len(prediction_high_conf_outliers)), np.zeros(len(prediction_high_conf_inliers))), axis = 0)
#             clf_prune_2 = LogisticRegression(random_state=0, penalty='l2', max_iter=max_iter).fit(scores_transformed[new_data_indexes], new_labels) 
# #             print("F-1 score from both LR and SVM:",metrics.f1_score(y, np.array([int(i) for i in clf_prune_2.predict_proba(scores_transformed)[:,1] > 0.5])))
# #             print('Coef from both LR and SVM: ', clf_prune_2.coef_[0])
#             combined_coef = clf_prune_2.coef_[0]  
#         else:
#             print('Coef from normal training: ', clf.coef_[0])
        combined_coef = clf.coef_[0]
#             print('Combined Coef: ',  combined_coef)

        if(np.max(coef_index_range) >= 2 or 
           ((np.max(combined_coef)/np.min(combined_coef) >= 1.1) and np.max(coef_index_range) >= 2)):
            if(len(set(combined_coef)) > 1):
                cur_clf_coef = combined_coef 
                cutoff = max(max(0, np.mean(combined_coef)-np.std(combined_coef)),min(combined_coef))
#                 print(cutoff)

                remain_indexes_after_cond = (cur_clf_coef > cutoff) #np.logical_and(cur_clf_coef > cutoff, abs(cur_clf_coef) > 0.01) # # 
                remain_params_tracking = remain_params_tracking[remain_indexes_after_cond]
                print(remain_params_tracking)
                remain_indexes_after_cond_expanded = []
                for i in range(0, len(coef_index_range)): #
                    s_e_range = coef_index_range[i,1]-coef_index_range[i,0]
                    s1, e1 = coef_index_range[i,0], coef_index_range[i,1]
                    s2, e2 = index_range[i,0], index_range[i,1]
                    saved_indexes = np.where(cur_clf_coef[s1:e1] > cutoff)[0]
                    for j in range(N_size):
                        remain_indexes_after_cond_expanded.extend(np.array(saved_indexes) + j * s_e_range + s2)

                new_coef_index_range_seq = []
                for i in range(0, len(coef_index_range)): #
                    s, e = coef_index_range[i,0], coef_index_range[i,1]
                    new_coef_index_range_seq.append(sum((remain_indexes_after_cond)[s:e]))

                coef_index_range = []
                index_range = []
                cur_sum = 0
                for i in range(0, len(new_coef_index_range_seq)):
                    coef_index_range.append([cur_sum, cur_sum + new_coef_index_range_seq[i]])
                    index_range.append([cur_sum * 6, 6 * (cur_sum + new_coef_index_range_seq[i])])
                    cur_sum += new_coef_index_range_seq[i]

                coef_index_range = np.array(coef_index_range)
                index_range = np.array(index_range)
                print(coef_index_range)
#                 print(index_range)

                L=L[:,remain_indexes_after_cond_expanded]
                scores_for_training = scores_for_training[:, remain_indexes_after_cond]
    if((len(last_training_data_indexes) == len(data_indexes)) and 
       (sum(last_training_data_indexes == data_indexes) == len(data_indexes)) and 
       (np.max(coef_index_range) < 2)):
        counter =  counter + 1
    else:
        counter = 0
    if(counter > 3):
        break
    last_training_data_indexes = data_indexes

##################################################################
Iteration = 0, L shape = (5473, 156)
num of outliers = 55
Training data shape:  (3843, 26)
Training data F-1 0.48
(3843, 10)
(3843,)
F-1 score from SVM: 0.492436974789916
Number of outliers by SVM: 645.0
F-1 score from LR: 0.38451612903225807
Number of outliers by LR:  215
[[1.        0.6164632]
 [0.6164632 1.       ]]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 23 25]
[[ 0 10]
 [10 20]
 [20 21]
 [21 22]]
##################################################################
Iteration = 1, L shape = (5473, 132)
num of outliers = 57
Training data shape:  (3844, 22)
Training data F-1 0.48684210526315785
(3844, 10)
(3844,)
F-1 score from SVM: 0.49372384937238495
Number of outliers by SVM: 652.0
F-1 score from LR: 0.3891752577319587
Number of outliers by LR:  216
[[1.         0.60554473]
 [0.60554473 1.        ]]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 25]
[[ 0 10]
 [10 20]
 [20 20]
 [20 21

F-1 score from SVM: 0.56120826709062
Number of outliers by SVM: 641.0
F-1 score from LR: 0.49907235621521334
Number of outliers by LR:  518
[[1.         0.69167663]
 [0.69167663 1.        ]]
[5]
[[0 1]
 [1 1]
 [1 1]
 [1 1]]
##################################################################
Iteration = 18, L shape = (5473, 6)
num of outliers = 323
Training data shape:  (4675, 1)
Training data F-1 0.6574923547400613
(4675, 10)
(4675,)
F-1 score from SVM: 0.5723472668810289
Number of outliers by SVM: 675.0
F-1 score from LR: 0.4909609895337773
Number of outliers by LR:  491
[[1.         0.71377166]
 [0.71377166 1.        ]]
##################################################################
Iteration = 19, L shape = (5473, 6)
num of outliers = 289
Training data shape:  (4660, 1)
Training data F-1 0.6402640264026402
(4660, 10)
(4660,)
F-1 score from SVM: 0.5788617886178863
Number of outliers by SVM: 664.0
F-1 score from LR: 0.5025432349949136
Number of outliers by LR:  423
[[1.         0.71

In [20]:
index_range = np.array([[0, 60], [60, 120], [120, 150], [150, 156]])
coef_index_range = np.array([[0, 10], [10, 20], [20, 25], [25, 26]])
coef_remain_index = range(156)

scores_for_training_indexes = []
for i in range(len(index_range)):
    start=index_range[i][0]
    temp_range = coef_index_range[i][1]-coef_index_range[i][0]
    scores_for_training_indexes  = scores_for_training_indexes + list(range(start, start+temp_range))
print(scores_for_training_indexes) 
scores_for_training = scores[:, np.array(scores_for_training_indexes)]
print(np.shape(scores_for_training))

transformer = RobustScaler().fit(scores_for_training)
scores_transformed = transformer.transform(scores_for_training)
training_data = scores_transformed[data_indexes]
print('Training data shape: ', np.shape(training_data))
training_data_F1.append(metrics.f1_score(y[data_indexes], labels))
print('Training data F-1', metrics.f1_score(y[data_indexes], labels))

transformer = RobustScaler().fit(X)
X_transformed = transformer.transform(X)
X_training_data = X_transformed[data_indexes]
print(np.shape(X_training_data))
print(np.shape(labels))

clf_X = SVC(gamma='auto', probability=True, random_state=0)
clf_X.fit(X_training_data, labels)
clf_predictions_X = clf_X.predict(X_transformed)
clf_predict_proba_X = clf_X.predict_proba(X_transformed)[:,1]
print("F-1 score from SVM:",metrics.f1_score(y, np.array([int(i) for i in clf_predict_proba_X > 0.5])))
cur_f1_scores.append(metrics.f1_score(y, np.array([int(i) for i in clf_predict_proba_X > 0.5])))

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
clf = LogisticRegression(random_state=0, penalty='l2', max_iter=max_iter).fit(training_data, labels) 
clf_predictions = clf.predict(scores_transformed)
clf_predict_proba = clf.predict_proba(scores_transformed)[:,1]
# LR_threshold = np.array(np.sort(clf_predict_proba)[::-1])[int(sum(clf_predictions_X))]
print("F-1 score from LR:",metrics.f1_score(y, np.array([int(i) for i in clf_predict_proba > LR_threshold])))
print('Number of outliers by LR: ', sum(np.array([int(i) for i in clf_predict_proba > LR_threshold])))


agreed_outlier_indexes = np.where(np.sum(L,axis=1)==np.shape(L)[1])[0]
agreed_inlier_indexes = np.where(np.sum(L,axis=1)==0)[0]

prediction_result_list.append(clf_predict_proba)
classifier_result_list.append(clf_predict_proba_X)

prediction_list.append(np.array([int(i) for i in clf_predictions]))

prediction_high_conf_outliers = np.intersect1d(np.where(prediction_result_list[-1] > high_confidence_threshold)[0],
                                               np.where(classifier_result_list[-1] > high_confidence_threshold)[0])
print('length of prediction_high_conf_outliers:' , len(prediction_high_conf_outliers))
prediction_high_conf_inliers = np.intersect1d(np.where(prediction_result_list[-1] < low_confidence_threshold)[0],
                                               np.where(classifier_result_list[-1] < low_confidence_threshold)[0])
print('length of prediction high conf inliers: ', len(prediction_high_conf_inliers))

temp_prediction = np.array([int(i) for i in prediction_result_list[-1] > LR_threshold])
temp_classifier = np.array([int(i) for i in classifier_result_list[-1] > 0.5])
prediction_classifier_disagree = np.where(temp_prediction != temp_classifier)[0]
print(np.corrcoef(clf_predict_proba,clf_predict_proba_X))

L = L_prev
remain_params_tracking = np.array(range(0,np.max(coef_index_range)))

if np.max(coef_index_range) >= 2:
    if(len(prediction_high_conf_outliers) > 0 and len(prediction_high_conf_inliers) > 0):
        new_data_indexes = np.concatenate((prediction_high_conf_outliers, prediction_high_conf_inliers), axis = 0)
        new_data_indexes = np.array([int(i) for i in new_data_indexes])
        new_labels = np.concatenate((np.ones(len(prediction_high_conf_outliers)), np.zeros(len(prediction_high_conf_inliers))), axis = 0)
        clf_prune_2 = LogisticRegression(random_state=0, penalty='l2', max_iter=max_iter).fit(scores_transformed[new_data_indexes], new_labels) 
        print("F-1 score from both LR and SVM:",metrics.f1_score(y, np.array([int(i) for i in clf_prune_2.predict_proba(scores_transformed)[:,1] > 0.5])))
        print('Coef from both LR and SVM: ', clf_prune_2.coef_[0])
        combined_coef = clf_prune_2.coef_[0]  
    else:
        print('Coef from normal training: ', clf.coef_[0])
        combined_coef = clf.coef_[0]
        print('Combined Coef: ',  combined_coef)

    if(np.max(coef_index_range) > 2 or 
       ((np.max(combined_coef)/np.min(combined_coef) >= 1.1) and np.max(coef_index_range) >= 2)):
        if(len(set(combined_coef)) > 1):
            cur_clf_coef = combined_coef 
            cutoff = max(max(0, np.mean(combined_coef)-np.std(combined_coef)),min(combined_coef))
            print(cutoff)

            remain_indexes_after_cond = (cur_clf_coef > cutoff) #np.logical_and(cur_clf_coef > cutoff, abs(cur_clf_coef) > 0.01) # # 
            remain_params_tracking = remain_params_tracking[remain_indexes_after_cond]
            print(remain_params_tracking)
            remain_indexes_after_cond_expanded = []
            for i in range(0, len(coef_index_range)): #
                s_e_range = coef_index_range[i,1]-coef_index_range[i,0]
                s1, e1 = coef_index_range[i,0], coef_index_range[i,1]
                s2, e2 = index_range[i,0], index_range[i,1]
                saved_indexes = np.where(cur_clf_coef[s1:e1] > cutoff)[0]
                for j in range(N_size):
                    remain_indexes_after_cond_expanded.extend(np.array(saved_indexes) + j * s_e_range + s2)

            new_coef_index_range_seq = []
            for i in range(0, len(coef_index_range)): #
                s, e = coef_index_range[i,0], coef_index_range[i,1]
                new_coef_index_range_seq.append(sum((remain_indexes_after_cond)[s:e]))

            coef_index_range = []
            index_range = []
            cur_sum = 0
            for i in range(0, len(new_coef_index_range_seq)):
                coef_index_range.append([cur_sum, cur_sum + new_coef_index_range_seq[i]])
                index_range.append([cur_sum * 6, 6 * (cur_sum + new_coef_index_range_seq[i])])
                cur_sum += new_coef_index_range_seq[i]

            coef_index_range = np.array(coef_index_range)
            index_range = np.array(index_range)
            print(coef_index_range)
            print(index_range)

            L=L[:,remain_indexes_after_cond_expanded]
            scores_for_training = scores_for_training[:, remain_indexes_after_cond]



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 120, 121, 122, 123, 124, 150]
(5473, 26)
Training data shape:  (4772, 26)
Training data F-1 0.6003210272873194
(4772, 10)
(4772,)
F-1 score from SVM: 0.5941780821917808
F-1 score from LR: 0.5048923679060664
Number of outliers by LR:  462
length of prediction_high_conf_outliers: 294
length of prediction high conf inliers:  4617
[[1.         0.78396188]
 [0.78396188 1.        ]]
F-1 score from both LR and SVM: 0.513888888888889
Coef from both LR and SVM:  [ 0.000736    0.1481505   0.30282109  0.45315032  0.51761562  0.58038926
  0.5933159   0.54739406  0.42410122  0.35553086 -0.05031496  0.01380261
  0.0438868   0.03950137  0.03704462  0.03240052  0.02769839  0.01778991
  0.01181254  0.01192402  0.15966538  0.16865174  0.16551874  0.16644401
  0.15644075  0.32235327]
0
[ 0  1  2  3  4  5  6  7  8  9 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]
[[ 0 10]
 [10 19]
 [19 24]
 [24 25]]
[[  0  60]
 [ 60 114]
 [114 144]
 [1

In [21]:
last_training_data_indexes = []
counter = 0

for i_range in range(0, 50):
    print("##################################################################")
    print('Iteration = {}, L shape = {}'.format(i_range, np.shape(L)))
    num_methods = np.shape(L)[1]

#     agree_outlier_indexes = (np.sum(L,axis=1)==np.shape(L)[1])
#     print('All agree, Number of outliers = {}'.format(sum(agree_outlier_indexes)))
#     agree_inlier_indexes = (np.sum(L,axis=1)==0)
#     print('All agree, Number of inliers = {}'.format(sum(agree_inlier_indexes)))

#     all_inlier_indexes = np.union1d(np.setdiff1d(np.where(agree_inlier_indexes)[0], prediction_high_conf_outliers), prediction_high_conf_inliers)
#     print('num of inliers = {}'.format(np.shape(all_inlier_indexes)[0]))
#     all_outlier_indexes = np.union1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)
#     print('num of outliers = {}'.format(np.shape(all_outlier_indexes)[0]))

#     disagree_indexes = np.where(np.logical_or(np.sum(L,axis = 1)==0, np.sum(L,axis = 1)==num_methods)==0)[0]

    ########################################################################

    agree_outlier_indexes = np.sum(L,axis=1)==np.shape(L)[1]
#     print('All agree, Number of outliers = {}'.format(sum(agree_outlier_indexes)))
    agree_inlier_indexes = np.sum(L,axis=1)==0
#     print('All agree, Number of inliers = {}'.format(sum(agree_inlier_indexes)))

    disagree_indexes = np.where(np.logical_or(np.sum(L,axis = 1)==0, np.sum(L,axis = 1)==num_methods)==0)[0]
    # print('Number of disagreed points = {}'.format(len(disagree_indexes)))
    # print('Number of disagreed points (true outliers) = {}'.format(sum(y[disagree_indexes] == 1)))
    # print('Number of disagreed points (true inliers) = {}'.format(sum(y[disagree_indexes] == 0)))

#     all_inlier_indexes = np.where(agree_inlier_indexes)[0]
    all_inlier_indexes = np.setdiff1d(np.where(agree_inlier_indexes)[0], prediction_high_conf_outliers)
    if len(prediction_high_conf_inliers) >0:
        all_inlier_indexes = np.intersect1d(np.setdiff1d(np.where(agree_inlier_indexes)[0], prediction_high_conf_outliers), prediction_high_conf_inliers)
#     print('num of inliers = {}'.format(np.shape(all_inlier_indexes)[0]))
#     all_outlier_indexes = np.union1d(np.where(agree_outlier_indexes)[0], self_agree_index_list)

#     if(len(np.intersect1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)) > 0 and
#       (len(np.union1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)) > 2000)):
#         all_outlier_indexes = np.intersect1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)
#     else:
    all_outlier_indexes = np.union1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)
#     if(len(all_outlier_indexes) > 1000):
#         all_outlier_indexes = np.random.RandomState(1).permutation(all_outlier_indexes)[:1000]
        
#     all_outlier_indexes = np.union1d(np.where(agree_outlier_indexes)[0], prediction_high_conf_outliers)
#     print('num of outliers = {}'.format(np.shape(all_outlier_indexes)[0]))
    all_inlier_indexes = np.setdiff1d(all_inlier_indexes, prediction_classifier_disagree)
    
    self_agree_index_list = []
    if((len(all_outlier_indexes) == 0) or (len(all_inlier_indexes)/ len(all_outlier_indexes) > 1000)):
        for i in range(0, len(index_range)):
            if(index_range[i,1]-index_range[i,0] <= 6):
                continue
            temp_index = disagree_indexes[np.where(np.sum(L[disagree_indexes][:,index_range[i,0]: index_range[i,1]], axis = 1)==(index_range[i,1]-index_range[i,0]))[0]]
            self_agree_index_list = np.union1d(self_agree_index_list, temp_index)
        self_agree_index_list = [int(i) for i in self_agree_index_list]
#     self_agree_index_list = np.random.RandomState(1).permutation(self_agree_index_list)[:500]
    all_outlier_indexes = np.union1d(all_outlier_indexes, self_agree_index_list)
    all_outlier_indexes = np.setdiff1d(all_outlier_indexes, prediction_classifier_disagree)
#     print('num of outliers = {}'.format(np.shape(all_outlier_indexes)[0]))
    
    
    from sklearn.preprocessing import RobustScaler
    from sklearn.preprocessing import StandardScaler
    data_indexes = np.concatenate((all_inlier_indexes, all_outlier_indexes), axis = 0)
    data_indexes = np.array([int(i) for i in data_indexes])
    labels = np.concatenate((np.zeros(len(all_inlier_indexes)), np.ones(len(all_outlier_indexes))), axis = 0)
    transformer = RobustScaler().fit(scores_for_training)
    scores_transformed = transformer.transform(scores_for_training)
    training_data = scores_transformed[data_indexes]
#     print('Training data shape: ', np.shape(training_data))
    training_data_F1.append(metrics.f1_score(y[data_indexes], labels))
#     print('Training data F-1', metrics.f1_score(y[data_indexes], labels))
    
    transformer = RobustScaler().fit(X)
    X_transformed = transformer.transform(X)
    X_training_data = X_transformed[data_indexes]
    print(np.shape(X_training_data))
    print(np.shape(labels))

    clf_X = SVC(gamma='auto', probability=True, random_state=0)
    clf_X.fit(X_training_data, labels)
    clf_predictions_X = clf_X.predict(X_transformed)
    clf_predict_proba_X = clf_X.predict_proba(X_transformed)[:,1]
    print("F-1 score from SVM:",metrics.f1_score(y, np.array([int(i) for i in clf_predict_proba_X > 0.5])))
    cur_f1_scores.append(metrics.f1_score(y, np.array([int(i) for i in clf_predict_proba_X > 0.5])))
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    clf = LogisticRegression(random_state=0, penalty='l2', max_iter=max_iter).fit(training_data, labels) 
    clf_predictions = clf.predict(scores_transformed)
    clf_predict_proba = clf.predict_proba(scores_transformed)[:,1]
    LR_threshold = np.array(np.sort(clf_predict_proba)[::-1])[int(sum(clf_predictions_X))]
    print("F-1 score from LR:",metrics.f1_score(y, np.array([int(i) for i in clf_predict_proba > LR_threshold])))
    print('Number of outliers by LR: ', sum(np.array([int(i) for i in clf_predict_proba > LR_threshold])))
    
    
    agreed_outlier_indexes = np.where(np.sum(L,axis=1)==np.shape(L)[1])[0]
    agreed_inlier_indexes = np.where(np.sum(L,axis=1)==0)[0]
        
    prediction_result_list.append(clf_predict_proba)
    classifier_result_list.append(clf_predict_proba_X)
    
    prediction_list.append(np.array([int(i) for i in clf_predictions]))
    print(np.corrcoef(clf_predict_proba,clf_predict_proba_X))
    
    prediction_high_conf_outliers = np.intersect1d(np.where(prediction_result_list[-1] > high_confidence_threshold)[0],
                                                   np.where(classifier_result_list[-1] > high_confidence_threshold)[0])
#     print('length of prediction_high_conf_outliers:' , len(prediction_high_conf_outliers))
    prediction_high_conf_inliers = np.intersect1d(np.where(prediction_result_list[-1] < low_confidence_threshold)[0],
                                                   np.where(classifier_result_list[-1] < low_confidence_threshold)[0])
#     print('length of prediction high conf inliers: ', len(prediction_high_conf_inliers))
    
    temp_prediction = np.array([int(i) for i in prediction_result_list[-1] > LR_threshold])
    temp_classifier = np.array([int(i) for i in classifier_result_list[-1] > 0.5])
    prediction_classifier_disagree = np.where(temp_prediction != temp_classifier)[0]
    
    if np.max(coef_index_range) >= 2:
        if(len(prediction_high_conf_outliers) > 0 and len(prediction_high_conf_inliers) > 0):
            new_data_indexes = np.concatenate((prediction_high_conf_outliers, prediction_high_conf_inliers), axis = 0)
            new_data_indexes = np.array([int(i) for i in new_data_indexes])
            new_labels = np.concatenate((np.ones(len(prediction_high_conf_outliers)), np.zeros(len(prediction_high_conf_inliers))), axis = 0)
            clf_prune_2 = LogisticRegression(random_state=0, penalty='l2', max_iter=max_iter).fit(scores_transformed[new_data_indexes], new_labels) 
#             print("F-1 score from both LR and SVM:",metrics.f1_score(y, np.array([int(i) for i in clf_prune_2.predict_proba(scores_transformed)[:,1] > 0.5])))
#             print('Coef from both LR and SVM: ', clf_prune_2.coef_[0])
            combined_coef = clf_prune_2.coef_[0]  
        else:
#             print('Coef from normal training: ', clf.coef_[0])
            combined_coef = clf.coef_[0]
#             print('Combined Coef: ',  combined_coef)

        if(np.max(coef_index_range) >= 2 or 
           ((np.max(combined_coef)/np.min(combined_coef) >= 1.1) and np.max(coef_index_range) >= 2)):
            if(len(set(combined_coef)) > 1):
                cur_clf_coef = combined_coef 
                cutoff = max(max(0, np.mean(combined_coef)-np.std(combined_coef)),min(combined_coef))
#                 print(cutoff)

                remain_indexes_after_cond = (cur_clf_coef > cutoff) #np.logical_and(cur_clf_coef > cutoff, abs(cur_clf_coef) > 0.01) # # 
                remain_params_tracking = remain_params_tracking[remain_indexes_after_cond]
                print(remain_params_tracking)
                remain_indexes_after_cond_expanded = []
                for i in range(0, len(coef_index_range)): #
                    s_e_range = coef_index_range[i,1]-coef_index_range[i,0]
                    s1, e1 = coef_index_range[i,0], coef_index_range[i,1]
                    s2, e2 = index_range[i,0], index_range[i,1]
                    saved_indexes = np.where(cur_clf_coef[s1:e1] > cutoff)[0]
                    for j in range(N_size):
                        remain_indexes_after_cond_expanded.extend(np.array(saved_indexes) + j * s_e_range + s2)

                new_coef_index_range_seq = []
                for i in range(0, len(coef_index_range)): #
                    s, e = coef_index_range[i,0], coef_index_range[i,1]
                    new_coef_index_range_seq.append(sum((remain_indexes_after_cond)[s:e]))

                coef_index_range = []
                index_range = []
                cur_sum = 0
                for i in range(0, len(new_coef_index_range_seq)):
                    coef_index_range.append([cur_sum, cur_sum + new_coef_index_range_seq[i]])
                    index_range.append([cur_sum * 6, 6 * (cur_sum + new_coef_index_range_seq[i])])
                    cur_sum += new_coef_index_range_seq[i]

                coef_index_range = np.array(coef_index_range)
                index_range = np.array(index_range)
                print(coef_index_range)
#                 print(index_range)

                L=L[:,remain_indexes_after_cond_expanded]
                scores_for_training = scores_for_training[:, remain_indexes_after_cond]
    if((len(last_training_data_indexes) == len(data_indexes)) and 
       (sum(last_training_data_indexes == data_indexes) == len(data_indexes)) and 
       (np.max(coef_index_range) < 2)):
        counter =  counter + 1
    else:
        counter = 0
    if(counter > 3):
        break
    last_training_data_indexes = data_indexes

##################################################################
Iteration = 0, L shape = (5473, 150)
(4106, 10)
(4106,)
F-1 score from SVM: 0.5628373168851195
F-1 score from LR: 0.5094482237339379
Number of outliers by LR:  763
[[1.         0.72055798]
 [0.72055798 1.        ]]
[ 1  2  3  4  5  6  7  8  9 12 13 14 15 16 17 18 20 21 22 23 24 25]
[[ 0  9]
 [ 9 16]
 [16 21]
 [21 22]]
##################################################################
Iteration = 1, L shape = (5473, 132)
(4302, 10)
(4302,)
F-1 score from SVM: 0.558641975308642
F-1 score from LR: 0.5079847908745247
Number of outliers by LR:  755
[[1.        0.7318044]
 [0.7318044 1.       ]]
[ 1  2  3  4  5  6  7  8  9 12 13 20 21 22 23 24 25]
[[ 0  9]
 [ 9 11]
 [11 16]
 [16 17]]
##################################################################
Iteration = 2, L shape = (5473, 102)
(4331, 10)
(4331,)
F-1 score from SVM: 0.5607333842627961
F-1 score from LR: 0.5132475397426193
Number of outliers by LR:  761
[[1.         0.

F-1 score from SVM: 0.6525490196078432
F-1 score from LR: 0.4924782264449723
Number of outliers by LR:  703
[[1.         0.69992274]
 [0.69992274 1.        ]]
##################################################################
Iteration = 27, L shape = (5473, 6)
(4688, 10)
(4688,)
F-1 score from SVM: 0.6552262090483619
F-1 score from LR: 0.4924782264449723
Number of outliers by LR:  703
[[1.         0.69654257]
 [0.69654257 1.        ]]
##################################################################
Iteration = 28, L shape = (5473, 6)
(4689, 10)
(4689,)
F-1 score from SVM: 0.6599378881987578
F-1 score from LR: 0.49014972419227737
Number of outliers by LR:  709
[[1.         0.69528068]
 [0.69528068 1.        ]]
##################################################################
Iteration = 29, L shape = (5473, 6)
(4689, 10)
(4689,)
F-1 score from SVM: 0.6599378881987578
F-1 score from LR: 0.49014972419227737
Number of outliers by LR:  709
[[1.         0.69528068]
 [0.69528068 1.       